# ETL Proyecto Individual ML Ops STEAM

In [2]:
# Importo las librerias que considero necesarias para hacer la extración de los Data Set
import json
import ast
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [75]:
# Ruta de ubicación para los datasets:
# 'E:\\AAADATOS\Henry\\AA_Data_Science\\MATERIAL_PI\\DATA_PI MLOps - STEAM\\Data_sets\\australian_user_reviews.json'
# 'E:\\AAADATOS\Henry\\AA_Data_Science\\MATERIAL_PI\\DATA_PI MLOps - STEAM\\Data_sets\\australian_users_items.json'
# 'E:\\AAADATOS\Henry\\AA_Data_Science\\MATERIAL_PI\\DATA_PI MLOps - STEAM\\Data_sets\\output_steam_games.json'

## Data Set australian_user_reviews.json

In [76]:
# Se inicializa una lista vacia donde alamcenare los datos del archivo .JSON
filas = []
# Se abre el archivo .JSON con el modulo 'with'
with open('E:\\AAADATOS\\Henry\\AA_Data_Science\\MATERIAL_PI\\DATA_PI MLOps - STEAM\\Data_sets\\australian_user_reviews.json', 'r', encoding='utf-8') as archivo:
    for line in archivo.readlines():
        filas.append(ast.literal_eval(line))

In [77]:
# Se inicializa el primer DataFrame a partir de la lista donde se extrajo el archivo .JSON
df_reviews = pd.DataFrame(filas)
df_reviews

,user_id,user_url,reviews
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'funny': '', 'posted': 'Posted November 5, 2..."
1,js41637,http://steamcommunity.com/id/js41637,"[{'funny': '', 'posted': 'Posted June 24, 2014..."
2,evcentric,http://steamcommunity.com/id/evcentric,"[{'funny': '', 'posted': 'Posted February 3.',..."
3,doctr,http://steamcommunity.com/id/doctr,"[{'funny': '', 'posted': 'Posted October 14, 2..."
4,maplemage,http://steamcommunity.com/id/maplemage,"[{'funny': '3 people found this review funny',..."
...,...,...,...
25794,76561198306599751,http://steamcommunity.com/profiles/76561198306...,"[{'funny': '', 'posted': 'Posted May 31.', 'la..."
25795,Ghoustik,http://steamcommunity.com/id/Ghoustik,"[{'funny': '', 'posted': 'Posted June 17.', 'l..."
25796,76561198310819422,http://steamcommunity.com/profiles/76561198310...,"[{'funny': '1 person found this review funny',..."
25797,76561198312638244,http://steamcommunity.com/profiles/76561198312...,"[{'funny': '', 'posted': 'Posted July 21.', 'l..."


Se evidencia que la columna "reviews" contiene listas como dato así que se tiene que proceder a expandirla

In [78]:
df_reviews = df_reviews.explode('reviews').reset_index() # Se redefine el DatFrame con una descomposición del campo para desanidarla y se reinicia el conteo del índice
df_reviews = df_reviews.drop(columns='index') # Se elimina el campo 'index'
df_reviews = pd.concat([df_reviews, pd.json_normalize(df_reviews['reviews'])], axis=1) # Se normalizan los datos JSON de la columna 'reviews' aplanando los datos, convirtiendolos en un formato tabular y concatenando de manera horizontal el DatFrame original con el del contendio de 'reviews' 
df_reviews.head()

,user_id,user_url,reviews,funny,posted,last_edited,item_id,helpful,recommend,review
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'funny': '', 'posted': 'Posted November 5, 20...",,"Posted November 5, 2011.",,1250,No ratings yet,True,Simple yet with great replayability. In my opi...
1,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'funny': '', 'posted': 'Posted July 15, 2011....",,"Posted July 15, 2011.",,22200,No ratings yet,True,It's unique and worth a playthrough.
2,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'funny': '', 'posted': 'Posted April 21, 2011...",,"Posted April 21, 2011.",,43110,No ratings yet,True,Great atmosphere. The gunplay can be a bit chu...
3,js41637,http://steamcommunity.com/id/js41637,"{'funny': '', 'posted': 'Posted June 24, 2014....",,"Posted June 24, 2014.",,251610,15 of 20 people (75%) found this review helpful,True,I know what you think when you see this title ...
4,js41637,http://steamcommunity.com/id/js41637,"{'funny': '', 'posted': 'Posted September 8, 2...",,"Posted September 8, 2013.",,227300,0 of 1 people (0%) found this review helpful,True,For a simple (it's actually not all that simpl...


### Se empieza a analizar y transformar los datos del DataFrame

In [79]:
# Información general del DataFrame
df_reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59333 entries, 0 to 59332
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   user_id      59333 non-null  object
 1   user_url     59333 non-null  object
 2   reviews      59305 non-null  object
 3   funny        59305 non-null  object
 4   posted       59305 non-null  object
 5   last_edited  59305 non-null  object
 6   item_id      59305 non-null  object
 7   helpful      59305 non-null  object
 8   recommend    59305 non-null  object
 9   review       59305 non-null  object
dtypes: object(10)
memory usage: 4.5+ MB


NULOS

In [80]:
# Conteo de Nulos
nulos = df_reviews.isna().sum()
nulos

user_id         0
user_url        0
reviews        28
funny          28
posted         28
last_edited    28
item_id        28
helpful        28
recommend      28
review         28
dtype: int64

In [81]:
# Visualizar los valores Nulos
df_nulos = df_reviews[df_reviews.isnull().any(axis=1)] # Se crea un nuevo DatFrame donde se filtra si hay algun valor nulo en la fila a partir del DataFrame orginal
df_nulos

,user_id,user_url,reviews,funny,posted,last_edited,item_id,helpful,recommend,review
137,gdxsd,http://steamcommunity.com/id/gdxsd,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
177,76561198094224872,http://steamcommunity.com/profiles/76561198094...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2559,76561198021575394,http://steamcommunity.com/profiles/76561198021...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10080,cmuir37,http://steamcommunity.com/id/cmuir37,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13767,Jaysteeny,http://steamcommunity.com/id/Jaysteeny,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15493,ML8989,http://steamcommunity.com/id/ML8989,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19184,76561198079215291,http://steamcommunity.com/profiles/76561198079...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20223,76561198079342142,http://steamcommunity.com/profiles/76561198079...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25056,76561198061996985,http://steamcommunity.com/profiles/76561198061...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
26257,76561198108286351,http://steamcommunity.com/profiles/76561198108...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


En base al DataFrame anterior se puede afirmar que todos los Nulos corresponden estas filas donde no esta la información relevante para mis objetivos con las consignas que se me asignaron por lo cual se procedera a ser eliminados

In [82]:
# Elimino las filas con valores nulos del DataFrame con el método 'dropna()'
df_reviews.dropna(inplace=True)
# Conteo de Nulos nuevamente para verificar
nuevos_nulos = df_reviews.isna().sum()
nuevos_nulos

user_id        0
user_url       0
reviews        0
funny          0
posted         0
last_edited    0
item_id        0
helpful        0
recommend      0
review         0
dtype: int64

De esta manera se han eliminado los valores Nulos del DataFrame hasta este punto de la transformación

#### Se procede a evaluar el contenido de los campos y determinar su relevancia dentro de los obejtivos

In [83]:
df_reviews.info()

<class 'pandas.core.frame.DataFrame'>
Index: 59305 entries, 0 to 59332
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   user_id      59305 non-null  object
 1   user_url     59305 non-null  object
 2   reviews      59305 non-null  object
 3   funny        59305 non-null  object
 4   posted       59305 non-null  object
 5   last_edited  59305 non-null  object
 6   item_id      59305 non-null  object
 7   helpful      59305 non-null  object
 8   recommend    59305 non-null  object
 9   review       59305 non-null  object
dtypes: object(10)
memory usage: 5.0+ MB


In [84]:
# Se crea una variable que contiene los valores unicos del campo 'funny'
valores_funny = df_reviews['funny'].unique() 
valores_funny

array(['', '3 people found this review funny',
       '1 person found this review funny',
       '2 people found this review funny',
       '5 people found this review funny',
       '17 people found this review funny',
       '524 people found this review funny',
       '4 people found this review funny',
       '12 people found this review funny',
       '6 people found this review funny',
       '7 people found this review funny',
       '103 people found this review funny',
       '16 people found this review funny',
       '34 people found this review funny',
       '9 people found this review funny',
       '45 people found this review funny',
       '11 people found this review funny',
       '97 people found this review funny',
       '207 people found this review funny',
       '718 people found this review funny',
       '23 people found this review funny',
       '100 people found this review funny',
       '8 people found this review funny',
       '19 people found this rev

In [85]:
# Se crea otra variable para contar cuantos valores hay que no sean strings vacios ("")
cant_funny = df_reviews[df_reviews['funny'] != ''].count()['funny']
cant_funny

8151

In [86]:
# Se hace una operación matematica para determinar que porcentaje representa los valores de funny completos dentro del conjunto del DataFrame
total_filas_df = 59305
print(f'La cantidad de valores que no son strings vacios en el campo "funny" representan un {round((cant_funny/total_filas_df)*100, 2)}%')

La cantidad de valores que no son strings vacios en el campo "funny" representan un 13.74%


Después del proceso anterior se puede determinar que los valores que se podrían considerar útiles del campo "funny" apenas representan un 13.74% dentro de todo el DataFrame por lo cual este campo en su conjunto se considerará presindible

In [87]:
# Se crea una variable que contiene los valores unicos del campo 'last_edited'
valores_last_edited = df_reviews['last_edited'].unique() 
valores_last_edited

array(['', 'Last edited December 5, 2015.',
       'Last edited November 25, 2013.', ...,
       'Last edited April 17, 2014.', 'Last edited May 10, 2014.',
       'Last edited August 15, 2014.'], dtype=object)

In [88]:
# Se crea otra variable para contar cuantos valores hay que no sean strings vacios ("")
cant_last_edited = df_reviews[df_reviews['last_edited'] != ''].count()['last_edited']
cant_last_edited

6140

In [89]:
# Se hace una operación matematica para determinar que porcentaje representa los valores de "last_edited" completos dentro del conjunto del DataFrame
print(f'La cantidad de valores que no son strings vacios en el campo "last_edited" representan un {round((cant_last_edited/total_filas_df)*100, 2)}%')

La cantidad de valores que no son strings vacios en el campo "last_edited" representan un 10.35%


Después del proceso anterior se puede determinar que los valores que se podrían considerar útiles del campo "last_edited" apenas representan un 10.35% dentro de todo el DataFrame por lo cual este campo en su conjunto se considerará presindible

In [90]:
# Se crea una variable que contiene los valores unicos del campo 'helpful'
valores_helpful = df_reviews['helpful'].unique() 
valores_helpful

array(['No ratings yet',
       '15 of 20 people (75%) found this review helpful',
       '0 of 1 people (0%) found this review helpful', ...,
       '21 of 34 people (62%) found this review helpful',
       '64 of 73 people (88%) found this review helpful',
       '24 of 34 people (71%) found this review helpful'], dtype=object)

In [91]:
# Se crea otra variable para contar cuantos valores hay que no sean "No ratings yet"
cant_helpful = df_reviews[df_reviews['helpful'] != 'No ratings yet'].count()['helpful']
cant_helpful

29137

In [92]:
# Se hace una operación matematica para determinar que porcentaje representa los valores de "helpful" completos dentro del conjunto del DataFrame
print(f'La cantidad de valores que no son "No ratings yet" vacios en el campo "helpful" representan un {round((cant_helpful/total_filas_df)*100, 2)}%')

La cantidad de valores que no son "No ratings yet" vacios en el campo "helpful" representan un 49.13%


Ahora que se posee mas información sobre el campo "helpful" se puede determinar que es presindible no solo por que su información relevante represente un poco menos del 50%, sino también por que esta información 'útil' seria dificil de cuantificar y llevar a escala además en los requerimientos se indica que el campo mas relevante para el objetivo es "review" por lo cual este campo tambien se presindé.

In [93]:
# Se crea una variable que contiene los valores unicos del campo 'recommend'
valores_recommend = df_reviews['recommend'].unique() 
valores_recommend

array([True, False], dtype=object)

Con el proceso anterior se cerciora que el campo "recommend" contiene información relevante en todas las filas del DataFrame

#### Fechas

In [94]:
# Se elimina la cadena 'Posted' de todos las filas del campo "posted"
df_reviews['posted'] = df_reviews['posted'].str.replace('Posted', '')
df_reviews.head(2)

,user_id,user_url,reviews,funny,posted,last_edited,item_id,helpful,recommend,review
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'funny': '', 'posted': 'Posted November 5, 20...",,"November 5, 2011.",,1250,No ratings yet,True,Simple yet with great replayability. In my opi...
1,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'funny': '', 'posted': 'Posted July 15, 2011....",,"July 15, 2011.",,22200,No ratings yet,True,It's unique and worth a playthrough.


In [95]:
# Se crea un nuevo campo llamado "Date" donde se almacenara el contenido de "posted" convertido a formato datetime
df_reviews['Date'] = pd.to_datetime(df_reviews['posted'], dayfirst=True, errors='coerce') # Con estos atributos se logra primero que se reconozca adecuadamente el formato de la fecha en el campo "posted" y segundo que lo que no se reconozca como datetime se convierta en una NaT "Not a Time"
df_reviews.head(2)

,user_id,user_url,reviews,funny,posted,last_edited,item_id,helpful,recommend,review,Date
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'funny': '', 'posted': 'Posted November 5, 20...",,"November 5, 2011.",,1250,No ratings yet,True,Simple yet with great replayability. In my opi...,2011-11-05
1,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'funny': '', 'posted': 'Posted July 15, 2011....",,"July 15, 2011.",,22200,No ratings yet,True,It's unique and worth a playthrough.,2011-07-15


In [96]:
# Se crea una variable para contabilizar la cantidad NaT's que hay en el nuevo campo "Date" del DataFrame
cant_Not_Date = df_reviews['Date'].isna().sum()
cant_Not_Date

10119

In [97]:
# Se crea un nuevo campor llamado "Year" donde se almacenara solamente el año de la review
df_reviews['Year'] = pd.to_datetime(df_reviews['Date']).dt.year
df_reviews.head(2)

,user_id,user_url,reviews,funny,posted,last_edited,item_id,helpful,recommend,review,Date,Year
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'funny': '', 'posted': 'Posted November 5, 20...",,"November 5, 2011.",,1250,No ratings yet,True,Simple yet with great replayability. In my opi...,2011-11-05,2011.0
1,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'funny': '', 'posted': 'Posted July 15, 2011....",,"July 15, 2011.",,22200,No ratings yet,True,It's unique and worth a playthrough.,2011-07-15,2011.0


Debido a que se cuenta con valores NaT en el campo "Date" el output del campo "Year" esta en formato float por lo cual se van a tranformar a formato "int"

In [98]:
# Se cuenta la cantidad de frecuencia de valores unicos que posee el campo "Year"
fecuencia_valores_Year = df_reviews['Year'].value_counts()
fecuencia_valores_Year

Year
2014.0    22111
2015.0    18470
2013.0     6792
2012.0     1215
2011.0      532
2010.0       66
Name: count, dtype: int64

In [99]:
# Se crea una variable para contabilizar la cantidad NaT's que hay en el nuevo campo "Year" del DataFrame
cant_Not_Year = df_reviews['Year'].isna().sum()
cant_Not_Year

10119

Con las tranformaciones anteriores se puede establecer que la cantidad de filas donde faltan unicamente el año en el campo "posted" son 10119 y es por esta razón que hay nulos al momento de transformalos a formatos de fecha

In [100]:
# Se reemplaza los valores NaT del campo "Year" por el año 2016
df_reviews['Year'].fillna(2016, inplace=True)
cant_Not_Year = df_reviews['Year'].isna().sum() # Se vuelve a contar los valores Nulos del campo
cant_Not_Year

0

In [101]:
# Se cuenta la cantidad de frecuencia de valores unicos que posee el campo "Year" nuevamente
fecuencia_valores_Year = df_reviews['Year'].value_counts()
fecuencia_valores_Year

Year
2014.0    22111
2015.0    18470
2016.0    10119
2013.0     6792
2012.0     1215
2011.0      532
2010.0       66
Name: count, dtype: int64

Esta decisión se justifica en un análisis externo de sobre Steam en la cual se encuentra que las reviews que no llevan el año en la fecha de posteo es por que fueron realizadas en el año en curso de esta manera como el último año que se tenía registro en el campo "posted" era el "2015" se asume que el año en curso es el "2016" y este pasa a reemplazar los valores Nulos que se tenian en estos campos de fechas

In [102]:
# Se transforma los datos del campo "Year" de tipo float a tipo int
df_reviews['Year'] = df_reviews['Year'].astype(int)
df_reviews.head(2)

,user_id,user_url,reviews,funny,posted,last_edited,item_id,helpful,recommend,review,Date,Year
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'funny': '', 'posted': 'Posted November 5, 20...",,"November 5, 2011.",,1250,No ratings yet,True,Simple yet with great replayability. In my opi...,2011-11-05,2011
1,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'funny': '', 'posted': 'Posted July 15, 2011....",,"July 15, 2011.",,22200,No ratings yet,True,It's unique and worth a playthrough.,2011-07-15,2011


In [103]:
# Se elimina el punto '.' de todos las filas del campo "posted"
df_reviews['posted'] = df_reviews['posted'].str.replace('.', '')
df_reviews.head(2)

,user_id,user_url,reviews,funny,posted,last_edited,item_id,helpful,recommend,review,Date,Year
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'funny': '', 'posted': 'Posted November 5, 20...",,"November 5, 2011",,1250,No ratings yet,True,Simple yet with great replayability. In my opi...,2011-11-05,2011
1,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'funny': '', 'posted': 'Posted July 15, 2011....",,"July 15, 2011",,22200,No ratings yet,True,It's unique and worth a playthrough.,2011-07-15,2011


Se procede a modificar el campo "posted" para poder añadir el año "2016" donde no lo tiene y de esta manera poder completar todos los valores del campo "Date"

In [104]:
patron_4_digitos = r'(\d{4})$' # Se crea una variable para identificar los valores donde no haya un patrón de cuatro dígitos consecutivos
df_reviews.loc[~df_reviews['posted'].str.contains(patron_4_digitos, regex=True), 'posted'] += ', 2016' # Se crea una máscara booleana que es True para las filas donde no hay un patrón de 4 números consecutivos al final

C:\Users\DavidssonPC\AppData\Local\Temp\ipykernel_7448\182169166.py:2: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df_reviews.loc[~df_reviews['posted'].str.contains(patron_4_digitos, regex=True), 'posted'] += ', 2016' # Se crea una máscara booleana que es True para las filas donde no hay un patrón de 4 números consecutivos al final


El siguiente paso es hacer nuevamente el campo "Date" pero esta vez  se completaran todos los campos así no quedando valores Nulos

In [105]:
# Se crea elimina el anterior campo "Date" y se transforma nuevamente
df_reviews = df_reviews.drop(columns='Date')
df_reviews['Date'] = pd.to_datetime(df_reviews['posted'], dayfirst=True, errors='coerce') # Con estos atributos se logra primero que se reconozca adecuadamente el formato de la fecha en el campo "posted" y segundo que lo que no se reconozca como datetime se convierta en una NaT "Not a Time"

In [106]:
# Se nuevamente la variable para contabilizar la cantidad NaT's que hay en el nuevo campo "Date" del DataFrame
cant_Not_Date = df_reviews['Date'].isna().sum()
cant_Not_Date

0

Esta vez ya no hay valores Nulos en los campos de fechas

#### Eliminar campos no relevantes

In [107]:
# Se crea una lista con el nombre de los campos que se van a eliminar del DataFrame
campos_out = ['user_url','reviews','funny','posted','last_edited','helpful']

In [108]:
df_reviews = df_reviews.drop(columns=campos_out)
df_reviews.head(2)

,user_id,item_id,recommend,review,Year,Date
0,76561197970982479,1250,True,Simple yet with great replayability. In my opi...,2011,2011-11-05
1,76561197970982479,22200,True,It's unique and worth a playthrough.,2011,2011-07-15


En este punto que ya se tienen los campos relevantes se procede a normalizar los nombre de estos

In [109]:
df_reviews = df_reviews.rename(columns={'user_id': 'User_Id'})
df_reviews = df_reviews.rename(columns={'item_id': 'Item_Id'})
df_reviews = df_reviews.rename(columns={'recommend': 'Recommend'})
df_reviews = df_reviews.rename(columns={'review': 'Review'})
df_reviews.head(2)

,User_Id,Item_Id,Recommend,Review,Year,Date
0,76561197970982479,1250,True,Simple yet with great replayability. In my opi...,2011,2011-11-05
1,76561197970982479,22200,True,It's unique and worth a playthrough.,2011,2011-07-15


#### Se crea la columna 'sentiment_analysis' aplicando análisis de sentimiento con NLP

Se importa la libreria nltk y se descarga el lexicon 'vader_lexicon' necesario para realizar el análisis de sentimientos

In [110]:
import nltk
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\DavidssonPC\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


Se crea la función que analizara los valores de la columna "Review" para darles un valor númerico según el análisis de sentimientos

In [111]:
# Se inicializa el analizador de sentimientos 
AS = SentimentIntensityAnalyzer()

# '0' si es malo 
# '1' si es neutral 
# '2' si es positivo

# Funcion para asignar valores de acuerdo a la escala
def puntaje_sentimiento(texto): # La variable de entrada de mi función seran los valores en formato str de la columna "Review"
    if pd.isnull(texto) or texto == '': # Evalua si el valor es un Nulo o un string vacío
        return 1 # Si es así retorna 1 que es un valor neutral
    elif isinstance(texto, str): # Verifica que el "texto" se una instancia del tipo str
        sentimiento = AS.polarity_scores(texto) # Se utiliza el analizador de sentimientos 'AS' para obtener un diccionario de puntajes de polarida y la función polarity_scores devuelve un diccionario que contiene puntajes para la polaridad positiva, negativa, neutra y un puntaje compuesto que resume la polaridad general del texto
        puntaje_compuesto = sentimiento['compound'] # Se extrae el puntaje compuesto del diccionario de sentimientos
        if puntaje_compuesto >= -0.05:
            return 2 # es positivo
        elif puntaje_compuesto <= -0.05:
            return 0 # es malo
        else:
            return 1 # es neutral
    else:
        return 1 # Si no cumple ninguna condición es neutral

In [112]:
# Se convierte la columna 'Review' a formato str
df_reviews['Review'] = df_reviews['Review'].astype(str)

# Se aplica la función puntaje_sentimiento a la columna 'Review' creando una nueva columna llamada 'Sentiment_Analysis' que contendra los puntajes de este análisis
df_reviews['Sentiment_Analysis'] = df_reviews['Review'].apply(puntaje_sentimiento)
df_reviews.head(2)

# Tardo 20.7 seg

,User_Id,Item_Id,Recommend,Review,Year,Date,Sentiment_Analysis
0,76561197970982479,1250,True,Simple yet with great replayability. In my opi...,2011,2011-11-05,2
1,76561197970982479,22200,True,It's unique and worth a playthrough.,2011,2011-07-15,2


Para este punto ya no es necesaria la columna "Review" y tal como se pide en las consignas esta ya cumplió su función y fue reemplazada por "Sentiment_Analysis" por lo caul va a ser eliminada

In [113]:
df_reviews = df_reviews.drop(columns=['Review']) # Se elimina el campo 'Review'
df_reviews.head(2)

,User_Id,Item_Id,Recommend,Year,Date,Sentiment_Analysis
0,76561197970982479,1250,True,2011,2011-11-05,2
1,76561197970982479,22200,True,2011,2011-07-15,2


In [114]:
df_reviews

,User_Id,Item_Id,Recommend,Year,Date,Sentiment_Analysis
0,76561197970982479,1250,True,2011,2011-11-05,2
1,76561197970982479,22200,True,2011,2011-07-15,2
2,76561197970982479,43110,True,2011,2011-04-21,2
3,js41637,251610,True,2014,2014-06-24,2
4,js41637,227300,True,2013,2013-09-08,2
...,...,...,...,...,...,...
59328,76561198312638244,70,True,2016,2016-07-10,2
59329,76561198312638244,362890,True,2016,2016-07-08,2
59330,LydiaMorley,273110,True,2016,2016-07-03,2
59331,LydiaMorley,730,True,2016,2016-07-20,2


### Se guarda el DataFrame en formatos mas ligeros

In [115]:
df_reviews.to_csv('user_reviews.csv', index=False) # Se guarda el DataFrame en formato CSV

In [116]:
# Se convierte el archivo CSV a parquet bajo la compresión "gzip"
ruta_csv_reviews = 'E:\\AAADATOS\\Henry\\AA_Data_Science\\MATERIAL_PI\\PI_ML_OPS_STEAM_DSFT17\\user_reviews.csv' # Se crea una variable con la ruta del archivo CSV
df_reviews_temp = pd.read_csv(ruta_csv_reviews) # Se lee ese CSV en un nuevo DataFrame temporal para segurar como estan los datos
df_reviews_temp

,User_Id,Item_Id,Recommend,Year,Date,Sentiment_Analysis
0,76561197970982479,1250,True,2011,2011-11-05,2
1,76561197970982479,22200,True,2011,2011-07-15,2
2,76561197970982479,43110,True,2011,2011-04-21,2
3,js41637,251610,True,2014,2014-06-24,2
4,js41637,227300,True,2013,2013-09-08,2
...,...,...,...,...,...,...
59300,76561198312638244,70,True,2016,2016-07-10,2
59301,76561198312638244,362890,True,2016,2016-07-08,2
59302,LydiaMorley,273110,True,2016,2016-07-03,2
59303,LydiaMorley,730,True,2016,2016-07-20,2


Se guarada el DataFrame temporal en formato comprimido gzip

In [117]:
df_reviews_temp.to_csv('user_reviews.gzip', compression='gzip', index=False)

In [118]:
# Se verifica que el archivo gzip comprimido pueda ser leido de manera efectiva y que no hayan problemas en los datos que este arroja
df_reviews_parquet = pd.read_csv('user_reviews.gzip', compression='gzip')
df_reviews_parquet

,User_Id,Item_Id,Recommend,Year,Date,Sentiment_Analysis
0,76561197970982479,1250,True,2011,2011-11-05,2
1,76561197970982479,22200,True,2011,2011-07-15,2
2,76561197970982479,43110,True,2011,2011-04-21,2
3,js41637,251610,True,2014,2014-06-24,2
4,js41637,227300,True,2013,2013-09-08,2
...,...,...,...,...,...,...
59300,76561198312638244,70,True,2016,2016-07-10,2
59301,76561198312638244,362890,True,2016,2016-07-08,2
59302,LydiaMorley,273110,True,2016,2016-07-03,2
59303,LydiaMorley,730,True,2016,2016-07-20,2


In [119]:
df_reviews_parquet.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59305 entries, 0 to 59304
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   User_Id             59305 non-null  object
 1   Item_Id             59305 non-null  int64 
 2   Recommend           59305 non-null  bool  
 3   Year                59305 non-null  int64 
 4   Date                59305 non-null  object
 5   Sentiment_Analysis  59305 non-null  int64 
dtypes: bool(1), int64(3), object(2)
memory usage: 2.3+ MB


## Data Set australian_users_items.json

In [3]:
# Se inicializa una lista vacia donde alamcenare los datso del archivo .JSON
filas = []
# Se abre el archivo .JSON con el modulo 'with'
with open('E:\\AAADATOS\\Henry\\AA_Data_Science\\MATERIAL_PI\\DATA_PI MLOps - STEAM\\Data_sets\\australian_users_items.json', 'r', encoding='utf-8') as archivo:
    for line in archivo.readlines():
        filas.append(ast.literal_eval(line))

In [4]:
# Se inicializa el DataFrame a partir de la lista donde se extrajo el archivo .JSON
df_items = pd.DataFrame(filas)
df_items

,user_id,items_count,steam_id,user_url,items
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
1,js41637,888,76561198035864385,http://steamcommunity.com/id/js41637,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
2,evcentric,137,76561198007712555,http://steamcommunity.com/id/evcentric,"[{'item_id': '1200', 'item_name': 'Red Orchest..."
3,Riot-Punch,328,76561197963445855,http://steamcommunity.com/id/Riot-Punch,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
4,doctr,541,76561198002099482,http://steamcommunity.com/id/doctr,"[{'item_id': '300', 'item_name': 'Day of Defea..."
...,...,...,...,...,...
88305,76561198323066619,22,76561198323066619,http://steamcommunity.com/profiles/76561198323...,"[{'item_id': '413850', 'item_name': 'CS:GO Pla..."
88306,76561198326700687,177,76561198326700687,http://steamcommunity.com/profiles/76561198326...,"[{'item_id': '11020', 'item_name': 'TrackMania..."
88307,XxLaughingJackClown77xX,0,76561198328759259,http://steamcommunity.com/id/XxLaughingJackClo...,[]
88308,76561198329548331,7,76561198329548331,http://steamcommunity.com/profiles/76561198329...,"[{'item_id': '304930', 'item_name': 'Unturned'..."


Se evidencia que la columna "items" contiene listas como dato así que se tiene que proceder a expandirla

In [5]:
df_items = df_items.explode('items').reset_index() # Se redefine el DatFrame con una descomposición del campo para desanidarla y se reinicia el conteo del índice
df_items = df_items.drop(columns='index') # Se elimina el campo 'index'
df_items = pd.concat([df_items, pd.json_normalize(df_items['items'])], axis=1) # Se normalizan los datos JSON de la columna 'reviews' aplanando los datos, convirtiendolos en un formato tabular y concatenando de manera horizontal el DatFrame original con el del contendio de 'reviews' 
df_items.head(2)

,user_id,items_count,steam_id,user_url,items,item_id,item_name,playtime_forever,playtime_2weeks
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'item_id': '10', 'item_name': 'Counter-Strike...",10,Counter-Strike,6.0,0.0
1,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'item_id': '20', 'item_name': 'Team Fortress ...",20,Team Fortress Classic,0.0,0.0


### Se empieza a analizar y transformar los datos del DataFrame

In [6]:
# Se verifica la información de los datos que contiene el DataFrame
df_items.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5170015 entries, 0 to 5170014
Data columns (total 9 columns):
 #   Column            Dtype  
---  ------            -----  
 0   user_id           object 
 1   items_count       int64  
 2   steam_id          object 
 3   user_url          object 
 4   items             object 
 5   item_id           object 
 6   item_name         object 
 7   playtime_forever  float64
 8   playtime_2weeks   float64
dtypes: float64(2), int64(1), object(6)
memory usage: 355.0+ MB


NULOS

In [7]:
# Conteo de Nulos
nulos = df_items.isna().sum()
nulos

user_id                 0
items_count             0
steam_id                0
user_url                0
items               16806
item_id             16806
item_name           16806
playtime_forever    16806
playtime_2weeks     16806
dtype: int64

In [8]:
# Se hace una operación matemática para determinar que poorcentaje de todo el DataSet representan esos valores Nulos
total_filas_df = 5170015
cant_nulos = 16806
print(f'La cantidad de valores nulos en el total del DataFrame es de {round(((cant_nulos/total_filas_df)*100), 2)}%')

La cantidad de valores nulos en el total del DataFrame es de 0.33%


Gracias a la operación anterior se determina que la cantidad de datos Nulos en el conjunto total del DataFrame representa una cantidad muy infima de los datos por lo cual se decide a ser eliminados

In [9]:
# Se eliminan los valores Nulos
df_items = df_items.dropna()

In [10]:
# Nuevamente se ha un conteo de Nulos
nulos = df_items.isna().sum()
nulos

user_id             0
items_count         0
steam_id            0
user_url            0
items               0
item_id             0
item_name           0
playtime_forever    0
playtime_2weeks     0
dtype: int64

In [11]:
# Se verifica nuevamente la información del DataFrame
df_items.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5153209 entries, 0 to 5170013
Data columns (total 9 columns):
 #   Column            Dtype  
---  ------            -----  
 0   user_id           object 
 1   items_count       int64  
 2   steam_id          object 
 3   user_url          object 
 4   items             object 
 5   item_id           object 
 6   item_name         object 
 7   playtime_forever  float64
 8   playtime_2weeks   float64
dtypes: float64(2), int64(1), object(6)
memory usage: 393.2+ MB


#### Eliminar campos no relevantes

In [12]:
# Se procede a eliminar campos no relevantes hasta este punto
df_items.drop(['user_url','items','playtime_2weeks'], axis=1, inplace=True)

Se considera el campo "playtime_2weeks" irrelevante ya que en las funciones que se especifican en la consigna solo piden las horas

In [13]:
df_items.shape # Se visualiza las dimensiones del DataFrame

(5153209, 6)

Debido al tamaño del DataFrame se considera eliminar las filas cuya hora sea igual a 0 ya que esto significa que el juego no se jugó por lo cual no sería considerado para valorar cada juego lo cual también necesario para las consignas osea las horas igual a 0 convierte las filas en irrelevantes

In [15]:
df_items = df_items[df_items['playtime_forever'] != 0.0] # Con este código reasigno el DatFrame sin las filas cuyos valores en 'playtime_forever' sea igual a 0.0 horas
df_items.shape

(3285246, 6)

In [18]:
# Se hace una operación para determinar en que porcentaje se redujo el tamaño del DataFrame
print(f'El tamaño del DataFrame se redujo en un {100 - (round(((3285246/5153209)*100), 2))}%')

El tamaño del DataFrame se redujo en un 36.25%


#### Transformación de datos

Se procede a cambiar el tipo de dato del campo "item_id" por tipo int

In [20]:
# Se crea una variable que contiene los valores unicos del campo 'item_id' para evalular que solo contenga numeros
valores_item_id = df_items['item_id'].unique() 
valores_item_id

array(['10', '30', '300', ..., '367090', '448540', '485270'], dtype=object)

In [21]:
# Ahora que se sabe que solo contiene valores númericos se procede a realizar el cambio de tipo de dato
df_items['item_id'] = df_items['item_id'].astype(int)

C:\Users\DavidssonPC\AppData\Local\Temp\ipykernel_7400\1433158164.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_items['item_id'] = df_items['item_id'].astype(int)


In [22]:
df_items.info() # Se verifica que haya cambiado el tipo de dato del campo 'item_id'

<class 'pandas.core.frame.DataFrame'>
Index: 3285246 entries, 0 to 5170013
Data columns (total 6 columns):
 #   Column            Dtype  
---  ------            -----  
 0   user_id           object 
 1   items_count       int64  
 2   steam_id          object 
 3   item_id           int32  
 4   item_name         object 
 5   playtime_forever  float64
dtypes: float64(1), int32(1), int64(1), object(3)
memory usage: 162.9+ MB


In [23]:
# Se procede a cambiar el tipo de dato del campo 'playtime_forever' a dato de tipo int ya que será necesario para un proceso futuro
df_items['playtime_forever'] = df_items['playtime_forever'].astype(int)
df_items.info() # Se evalúa que efectivamente se hayan cambiado los tipos de datos

<class 'pandas.core.frame.DataFrame'>
Index: 3285246 entries, 0 to 5170013
Data columns (total 6 columns):
 #   Column            Dtype 
---  ------            ----- 
 0   user_id           object
 1   items_count       int64 
 2   steam_id          object
 3   item_id           int32 
 4   item_name         object
 5   playtime_forever  int32 
dtypes: int32(2), int64(1), object(3)
memory usage: 150.4+ MB


C:\Users\DavidssonPC\AppData\Local\Temp\ipykernel_7400\1644264538.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_items['playtime_forever'] = df_items['playtime_forever'].astype(int)


In [24]:
df_items.head(2)

,user_id,items_count,steam_id,item_id,item_name,playtime_forever
0,76561197970982479,277,76561197970982479,10,Counter-Strike,6
2,76561197970982479,277,76561197970982479,30,Day of Defeat,7


En este punto que ya se tienen los campos relevantes se procede a normalizar los nombre de estos

In [25]:
df_items = df_items.rename(columns={'user_id': 'User_Id'})
df_items = df_items.rename(columns={'items_count': 'Items_Count'})
df_items = df_items.rename(columns={'steam_id': 'Steam_Id'})
df_items = df_items.rename(columns={'item_id': 'Item_Id'})
df_items = df_items.rename(columns={'item_name': 'Item_Name'})
df_items = df_items.rename(columns={'playtime_forever': 'Playtime_Forever_Hours'})
df_items.head(2)

,User_Id,Items_Count,Steam_Id,Item_Id,Item_Name,Playtime_Forever_Hours
0,76561197970982479,277,76561197970982479,10,Counter-Strike,6
2,76561197970982479,277,76561197970982479,30,Day of Defeat,7


### Se guarda el DataFrame en formatos mas ligeros

In [26]:
df_items.to_csv('user_items.csv', index=False) # Se guarda el DataFrame en formato CSV)

In [28]:
# Se convierte el archivo CSV a parquet bajo la compresión "gzip"
ruta_csv_items = 'E:\\AAADATOS\\Henry\\AA_Data_Science\\MATERIAL_PI\\PI_ML_OPS_STEAM_DSFT17\\user_items.csv' # Se crea una variable con la ruta del archivo CSV
df_items_temp = pd.read_csv(ruta_csv_items) # Se lee ese CSV en un nuevo DataFrame temporal para segurar como estan los datos
df_items_temp

,User_Id,Items_Count,Steam_Id,Item_Id,Item_Name,Playtime_Forever_Hours
0,76561197970982479,277,76561197970982479,10,Counter-Strike,6
1,76561197970982479,277,76561197970982479,30,Day of Defeat,7
2,76561197970982479,277,76561197970982479,300,Day of Defeat: Source,4733
3,76561197970982479,277,76561197970982479,240,Counter-Strike: Source,1853
4,76561197970982479,277,76561197970982479,3830,Psychonauts,333
...,...,...,...,...,...,...
3285241,76561198329548331,7,76561198329548331,304930,Unturned,677
3285242,76561198329548331,7,76561198329548331,227940,Heroes & Generals,43
3285243,76561198329548331,7,76561198329548331,388490,One Way To Die: Steam Edition,3
3285244,76561198329548331,7,76561198329548331,521570,You Have 10 Seconds 2,4


Se guarada el DataFrame temporal en formato comprimido gzip

In [29]:
df_items_temp.to_csv('user_items.gzip', compression='gzip', index=False)

In [30]:
# Se verifica que el archivo gzip comprimido pueda ser leido de manera efectiva y que no hayan problemas en los datos que este arroja
df_items_parquet = pd.read_csv('user_items.gzip', compression='gzip')
df_items_parquet

,User_Id,Items_Count,Steam_Id,Item_Id,Item_Name,Playtime_Forever_Hours
0,76561197970982479,277,76561197970982479,10,Counter-Strike,6
1,76561197970982479,277,76561197970982479,30,Day of Defeat,7
2,76561197970982479,277,76561197970982479,300,Day of Defeat: Source,4733
3,76561197970982479,277,76561197970982479,240,Counter-Strike: Source,1853
4,76561197970982479,277,76561197970982479,3830,Psychonauts,333
...,...,...,...,...,...,...
3285241,76561198329548331,7,76561198329548331,304930,Unturned,677
3285242,76561198329548331,7,76561198329548331,227940,Heroes & Generals,43
3285243,76561198329548331,7,76561198329548331,388490,One Way To Die: Steam Edition,3
3285244,76561198329548331,7,76561198329548331,521570,You Have 10 Seconds 2,4


In [31]:
df_items_parquet.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3285246 entries, 0 to 3285245
Data columns (total 6 columns):
 #   Column                  Dtype 
---  ------                  ----- 
 0   User_Id                 object
 1   Items_Count             int64 
 2   Steam_Id                int64 
 3   Item_Id                 int64 
 4   Item_Name               object
 5   Playtime_Forever_Hours  int64 
dtypes: int64(4), object(2)
memory usage: 150.4+ MB


## Data Set output_steam_games.json

In [163]:
# Se inicializa una lista vacia donde alamcenare los datso del archivo .JSON
filas = []
# Se abre el archivo .JSON con el modulo 'with'
with open('E:\\AAADATOS\\Henry\\AA_Data_Science\\MATERIAL_PI\\DATA_PI MLOps - STEAM\\Data_sets\\output_steam_games.json', 'r', encoding='utf-8') as archivo:
    for line in archivo.readlines():
        filas.append(json.loads(line))

In [164]:
# Se inicializa el DataFrame a partir de la lista donde se extrajo el archivo .JSON
df_games = pd.DataFrame(filas)
df_games

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
120440,Ghost_RUS Games,"[Casual, Indie, Simulation, Strategy]",Colony On Mars,Colony On Mars,http://store.steampowered.com/app/773640/Colon...,2018-01-04,"[Strategy, Indie, Casual, Simulation]",http://steamcommunity.com/app/773640/reviews/?...,"[Single-player, Steam Achievements]",1.99,False,773640,"Nikita ""Ghost_RUS"""
120441,Sacada,"[Casual, Indie, Strategy]",LOGistICAL: South Africa,LOGistICAL: South Africa,http://store.steampowered.com/app/733530/LOGis...,2018-01-04,"[Strategy, Indie, Casual]",http://steamcommunity.com/app/733530/reviews/?...,"[Single-player, Steam Achievements, Steam Clou...",4.99,False,733530,Sacada
120442,Laush Studio,"[Indie, Racing, Simulation]",Russian Roads,Russian Roads,http://store.steampowered.com/app/610660/Russi...,2018-01-04,"[Indie, Simulation, Racing]",http://steamcommunity.com/app/610660/reviews/?...,"[Single-player, Steam Achievements, Steam Trad...",1.99,False,610660,Laush Dmitriy Sergeevich
120443,SIXNAILS,"[Casual, Indie]",EXIT 2 - Directions,EXIT 2 - Directions,http://store.steampowered.com/app/658870/EXIT_...,2017-09-02,"[Indie, Casual, Puzzle, Singleplayer, Atmosphe...",http://steamcommunity.com/app/658870/reviews/?...,"[Single-player, Steam Achievements, Steam Cloud]",4.99,False,658870,"xropi,stev3ns"


### Se empieza a analizar y transformar los datos del DataFrame

In [165]:
# Se verifica la información de los datos que contiene el DataFrame
df_games.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120445 entries, 0 to 120444
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   publisher     24083 non-null  object
 1   genres        28852 non-null  object
 2   app_name      32133 non-null  object
 3   title         30085 non-null  object
 4   url           32135 non-null  object
 5   release_date  30068 non-null  object
 6   tags          31972 non-null  object
 7   reviews_url   32133 non-null  object
 8   specs         31465 non-null  object
 9   price         30758 non-null  object
 10  early_access  32135 non-null  object
 11  id            32133 non-null  object
 12  developer     28836 non-null  object
dtypes: object(13)
memory usage: 11.9+ MB


#### Eliminar campos no relevantes

In [166]:
# Se crea una lista con el nombre de los campos que se van a eliminar del DataFrame
campos_eliminar = ['app_name','url','reviews_url','specs','price','early_access']

In [167]:
df_games = df_games.drop(columns=campos_eliminar)
df_games.head(2)

,publisher,genres,title,release_date,tags,id,developer
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN


##### Se decidio eliminar estos campos debido a diversas razones que van desde que tenian información poco relevante, información que se cumplia con la de otro campo o que no se considera necesaria para cumplir con las consignas que se piden

NULOS

In [168]:
# Conteo de Nulos
nulos = df_games.isna().sum()
nulos

publisher       96362
genres          91593
title           90360
release_date    90377
tags            88473
id              88312
developer       91609
dtype: int64

Se hace notorio que hay filas que estan completamente llenas de NaN's así que se procedera a eliminar esas primero

In [169]:
df_games = df_games.dropna(how='all') # Se le indica que elimine solo las filas que contienen NaN en todos los campos

In [170]:
# Se vuelve a contar los Nulos
nulos = df_games.isna().sum()
nulos

publisher       8051
genres          3282
title           2049
release_date    2066
tags             162
id                 1
developer       3298
dtype: int64

In [171]:
# Se revisa el nuevo tamaño del DataFrame para ver en cuanto se redujo su contenido
df_games.shape

(32134, 7)

In [172]:
print(f'Eliminando solamente las filas completas de valores Nulos se redujo el Data Set en un {100 - (round(((32134/120445)*100) ,2))}%')

Eliminando solamente las filas completas de valores Nulos se redujo el Data Set en un 73.32%


Se elimina la fila donde existe un valor Nulo en el campo "id"

In [173]:
df_games.dropna(subset=["id"], inplace=True)

##### Siguiendo las consignas de las funciónes de colsulta se puede determinar que los campos "genres" y "developer" son crusiales por lo cual se procedera a eliminar las filas donde estos tengan valores Nulos

In [174]:
df_games.dropna(subset=["genres"], inplace=True) # Se eliminan los nulos del campo 'genres'
df_games.dropna(subset=["developer"], inplace=True) # Se eliminan los nulos del campo 'developer'

In [175]:
# Se revisa cuantos valores nulos se tiene hasta el momento
nulos = df_games.isna().sum()
nulos

publisher       4780
genres             0
title              0
release_date      18
tags              24
id                 0
developer          0
dtype: int64

In [176]:
df_games.shape # Se analiza el número de filas que tiene el DataSet hasta el momento

(28681, 7)

En base a lo anterior se considera que la cantidad de valores Nulos del campo "release_date" son infimos dentro del conjunto del DataFrame por lo cual se procede a ser eliminados, es de aclarar que la información de este campo tambien es necesaria para las consginas

In [177]:
df_games.dropna(subset=["release_date"], inplace=True) # Se eliminan los nulos del campo 'release_date'
df_games.shape

(28663, 7)

In [178]:
# Se revisa cuantos valores nulos se tiene hasta el momento
nulos = df_games.isna().sum()
nulos

publisher       4779
genres             0
title              0
release_date       0
tags              24
id                 0
developer          0
dtype: int64

La cantidad de Nulos que quedan en el campo "publisher" es considerable además de ser un campo que podría no ser presendible para las funciones se decide dejarlo y reemplazar los valores Nulos por un string que indique "No Data" de esta manera "N/D" y lo mismo para el campo "tags"

In [179]:
df_games.info() 

<class 'pandas.core.frame.DataFrame'>
Index: 28663 entries, 88310 to 120443
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   publisher     23884 non-null  object
 1   genres        28663 non-null  object
 2   title         28663 non-null  object
 3   release_date  28663 non-null  object
 4   tags          28639 non-null  object
 5   id            28663 non-null  object
 6   developer     28663 non-null  object
dtypes: object(7)
memory usage: 1.7+ MB


Se observa que el tipo de dato de todos los campos esta en "str"

In [180]:
# Se reemplaza los valores Nulos del campo 'tags' por una la siguiente lista '[N/D]'
df_games['tags'] = df_games['tags'].fillna('[N/D]')

In [181]:
# Se verifica que ya no hayan valores Nulos en el campo 'tags'
nulos = df_games.isna().sum()
nulos

publisher       4779
genres             0
title              0
release_date       0
tags               0
id                 0
developer          0
dtype: int64

In [182]:
# Se reemplaza los valores Nulos del campo 'publisher' por una el siguiente string 'N/D'
df_games['publisher'] = df_games['publisher'].fillna('N/D')

In [183]:
# Se verifica que ya no hayan valores Nulos en el campo 'publisher'
nulos = df_games.isna().sum()
nulos

publisher       0
genres          0
title           0
release_date    0
tags            0
id              0
developer       0
dtype: int64

Hasta este punto ya no hay valores Nulos en el Data Frame

##### Se identifica una complejidad en el campo "release_date" por lo cual se procede a ser tranformado

In [184]:
# Se realiza un filtrado de los valores que no coinciden con el patrón de fecha
import re
patron = re.compile(r'^\d{4}-\d{2}-\d{2}$')
filas_no_patron = df_games[~df_games['release_date'].astype(str).str.match(patron)]
filas_no_patron

,publisher,genres,title,release_date,tags,id,developer
88320,Qucheza,"[Action, Indie, Simulation, Early Access]",Uncanny Islands,Soon..,"[Early Access, Action, Indie, Simulation, Surv...",768570,Qucheza
88353,Breakout Studio,"[Indie, Massively Multiplayer, RPG, Early Access]",Life of Rome,2017,"[Early Access, Massively Multiplayer, RPG, Ind...",381620,Breakout Studio
88354,zha7,"[Adventure, Free to Play, RPG, Early Access]",Idle Adventure,Beta测试已开启,"[Early Access, Free to Play, Adventure, RPG]",714020,"zha7,Phaedruns,Yingke Wu"
88560,Erik Svedäng,"[Adventure, Indie]",Blueberry Garden,Jun 2009,"[Adventure, Indie, Short, Puzzle]",29160,Erik Svedäng
88816,Paradox Interactive,"[RPG, Strategy]",Age of Wonders II: The Wizard's Throne,Oct 2010,"[Strategy, RPG, Turn-Based Strategy, Fantasy, ...",61510,Triumph Studios
...,...,...,...,...,...,...,...
120392,Fat Dog Games,"[Action, Adventure, Indie, Simulation, Strategy]",War Builder League,15.01.2018,"[Action, Indie, Simulation, Adventure, Strateg...",716800,Cyber Prophet
120393,Wrecked Angle Studios,"[Action, Casual, Indie, Early Access]",Chromalocity,"Q1, 2018","[Early Access, Action, Indie, Casual, Platform...",718250,Wrecked Angle Studios
120397,"Greyson Richey, Nicholas Lives","[Action, Casual, Indie, Simulation]",Lonely Astronaut,Coming Soon,"[Casual, Action, Indie, Simulation]",755830,"Greyson Richey,Nicholas Lives"
120407,Lunar Giant Studios,"[Action, Indie]",Tetrapulse,2016,"[Indie, Action, Co-op]",250440,The Amiable


Se descubre que hay una cantidad de filas cuyos datos de "release_date" son muy variados o no contienen información exacta de alguna fecha haciendolo dificil de normalizar por lo cual se procede a eliminar estas filas

In [185]:
# Se eliminan las filas que no coinciden con el patron de fecha de 'release_date'
df_games = df_games.drop(filas_no_patron.index)
df_games.shape

(28378, 7)

In [187]:
# Se verifica que no hayan valores mas largos como valor en'realease_rate' que no se puedan convertir a una fecha
longitud_no_deseada = df_games[df_games['release_date'].astype(str).apply(len) != 10]
longitud_no_deseada

,publisher,genres,title,release_date,tags,id,developer


In [188]:
# Se eliminan los últimos 6 dijitos del campo 'release_date' para que solamente quede los dijitos del año
df_games['release_date'] = df_games['release_date'].astype(str).str[:4]
df_games.head()

,publisher,genres,title,release_date,tags,id,developer
88310,Kotoshiro,"[Action, Casual, Indie, Simulation, Strategy]",Lost Summoner Kitty,2018,"[Strategy, Action, Indie, Casual, Simulation]",761140,Kotoshiro
88311,"Making Fun, Inc.","[Free to Play, Indie, RPG, Strategy]",Ironbound,2018,"[Free to Play, Strategy, Indie, RPG, Card Game...",643980,Secret Level SRL
88312,Poolians.com,"[Casual, Free to Play, Indie, Simulation, Sports]",Real Pool 3D - Poolians,2017,"[Free to Play, Simulation, Sports, Casual, Ind...",670290,Poolians.com
88313,彼岸领域,"[Action, Adventure, Casual]",弹炸人2222,2017,"[Action, Adventure, Casual]",767400,彼岸领域
88315,Trickjump Games Ltd,"[Action, Adventure, Simulation]",Battle Royale Trainer,2018,"[Action, Adventure, Simulation, FPS, Shooter, ...",772540,Trickjump Games Ltd


In [189]:
# Se verifica que los únicos valores del campo sean cadenas que coincidan con años
fecuencia_valores_date = df_games['release_date'].unique()
fecuencia_valores_date

array(['2018', '2017', '1997', '1998', '2016', '2006', '2005', '2003',
       '2007', '2002', '2000', '1995', '1996', '1994', '2001', '1993',
       '2004', '1999', '2008', '2009', '1992', '1989', '2010', '2011',
       '2013', '2012', '2014', '1983', '1984', '2015', '1990', '1988',
       '1991', '1987', '1986', '2021', '2019', '1985'], dtype=object)

Se reinicia el conteo del index del Data Frame

In [190]:
df_games = df_games.reset_index()
df_games = df_games.drop(columns='index')

Se procede a Normalizar los nombres de los campos

In [191]:
df_games = df_games.rename(columns={'publisher': 'Publisher'})
df_games = df_games.rename(columns={'genres': 'Genres'})
df_games = df_games.rename(columns={'title': 'Title'})
df_games = df_games.rename(columns={'release_date': 'Year'})
df_games = df_games.rename(columns={'tags': 'Tags'})
df_games = df_games.rename(columns={'id': 'Item_Id'})
df_games = df_games.rename(columns={'developer': 'Developer'})
df_games.head(2)

,Publisher,Genres,Title,Year,Tags,Id,Developer
0,Kotoshiro,"[Action, Casual, Indie, Simulation, Strategy]",Lost Summoner Kitty,2018,"[Strategy, Action, Indie, Casual, Simulation]",761140,Kotoshiro
1,"Making Fun, Inc.","[Free to Play, Indie, RPG, Strategy]",Ironbound,2018,"[Free to Play, Strategy, Indie, RPG, Card Game...",643980,Secret Level SRL


#### Se transforma los tipos de datos de los valores 

In [192]:
# Se cambia el tipo de dato del campo 'Release_Date' por tipo de dato Date Time
df_games['Year'] = df_games['Year'].astype(int)
df_games.head(2)

,Publisher,Genres,Title,Year,Tags,Id,Developer
0,Kotoshiro,"[Action, Casual, Indie, Simulation, Strategy]",Lost Summoner Kitty,2018,"[Strategy, Action, Indie, Casual, Simulation]",761140,Kotoshiro
1,"Making Fun, Inc.","[Free to Play, Indie, RPG, Strategy]",Ironbound,2018,"[Free to Play, Strategy, Indie, RPG, Card Game...",643980,Secret Level SRL


In [193]:
# Se transforma los datos del campo 'Item_Id' de tipo str a tipo int
df_games['Item_Id'] = df_games['Item_Id'].astype(int)
df_games.head(2)

,Publisher,Genres,Title,Year,Tags,Id,Developer
0,Kotoshiro,"[Action, Casual, Indie, Simulation, Strategy]",Lost Summoner Kitty,2018,"[Strategy, Action, Indie, Casual, Simulation]",761140,Kotoshiro
1,"Making Fun, Inc.","[Free to Play, Indie, RPG, Strategy]",Ironbound,2018,"[Free to Play, Strategy, Indie, RPG, Card Game...",643980,Secret Level SRL


In [194]:
# Se transforma los datos de los campos 'Genres' y 'Tags' de tipo str a tipo str
df_games['Genres'] = df_games['Genres'].astype(str)
df_games['Tags'] = df_games['Tags'].astype(str)
df_games.head(2)

,Publisher,Genres,Title,Year,Tags,Id,Developer
0,Kotoshiro,"['Action', 'Casual', 'Indie', 'Simulation', 'S...",Lost Summoner Kitty,2018,"['Strategy', 'Action', 'Indie', 'Casual', 'Sim...",761140,Kotoshiro
1,"Making Fun, Inc.","['Free to Play', 'Indie', 'RPG', 'Strategy']",Ironbound,2018,"['Free to Play', 'Strategy', 'Indie', 'RPG', '...",643980,Secret Level SRL


In [195]:
# Se verifica estos cambios de tipos de dato
df_games.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28378 entries, 0 to 28377
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Publisher  28378 non-null  object
 1   Genres     28378 non-null  object
 2   Title      28378 non-null  object
 3   Year       28378 non-null  int32 
 4   Tags       28378 non-null  object
 5   Id         28378 non-null  int32 
 6   Developer  28378 non-null  object
dtypes: int32(2), object(5)
memory usage: 1.3+ MB


In [196]:
# Se elimina los corchetes del principio y el final de las cadenas
df_games['Genres'] = df_games['Genres'].str.strip('[]')
df_games['Tags'] = df_games['Tags'].str.strip('[]')
df_games.head(2)

,Publisher,Genres,Title,Year,Tags,Id,Developer
0,Kotoshiro,"'Action', 'Casual', 'Indie', 'Simulation', 'St...",Lost Summoner Kitty,2018,"'Strategy', 'Action', 'Indie', 'Casual', 'Simu...",761140,Kotoshiro
1,"Making Fun, Inc.","'Free to Play', 'Indie', 'RPG', 'Strategy'",Ironbound,2018,"'Free to Play', 'Strategy', 'Indie', 'RPG', 'C...",643980,Secret Level SRL


In [197]:
# Se elimina la la comilla "'" de todos las filas de los campos  'Genres' y 'Tags'
df_games['Genres'] = df_games['Genres'].str.replace("'", '')
df_games['Tags'] = df_games['Tags'].str.replace("'", '')
df_games.head(2)

,Publisher,Genres,Title,Year,Tags,Id,Developer
0,Kotoshiro,"Action, Casual, Indie, Simulation, Strategy",Lost Summoner Kitty,2018,"Strategy, Action, Indie, Casual, Simulation",761140,Kotoshiro
1,"Making Fun, Inc.","Free to Play, Indie, RPG, Strategy",Ironbound,2018,"Free to Play, Strategy, Indie, RPG, Card Game,...",643980,Secret Level SRL


In [198]:
# Se crea una variable para contar cuantos valores hay que sean strings ("") en el campo 'Generes'
cant_generes = df_games[df_games['Genres'] == ''].count()['Genres']
cant_generes

0

In [199]:
# Se crea una variable para contar cuantos valores hay que sean strings ("") en el campo 'Tags'
cant_tags = df_games[df_games['Tags'] == ''].count()['Tags']
cant_tags

0

De esta manera se verifica que ya no hay filas con datos irelevantes hasta el momento

##### Se decide tomar como relevante el contenido del campo "Tags" para las consultas sobre los generos por lo cual se unira al campo "Genres" creando un nuevo campo llamado "Genres_Plus" donde solo aparezcan los valores únicos resultantes 

In [200]:
df_games['Genres_Plus'] = (df_games['Genres'].str.split(', ') + df_games['Tags'].str.split(', ')).apply(set).apply(list).apply(', '.join)
df_games

,Publisher,Genres,Title,Year,Tags,Id,Developer,Genres_Plus
0,Kotoshiro,"Action, Casual, Indie, Simulation, Strategy",Lost Summoner Kitty,2018,"Strategy, Action, Indie, Casual, Simulation",761140,Kotoshiro,"Simulation, Strategy, Casual, Action, Indie"
1,"Making Fun, Inc.","Free to Play, Indie, RPG, Strategy",Ironbound,2018,"Free to Play, Strategy, Indie, RPG, Card Game,...",643980,Secret Level SRL,"2D, Replay Value, RPG, Turn-Based, Free to Pla..."
2,Poolians.com,"Casual, Free to Play, Indie, Simulation, Sports",Real Pool 3D - Poolians,2017,"Free to Play, Simulation, Sports, Casual, Indi...",670290,Poolians.com,"Sports, Simulation, Free to Play, Multiplayer,..."
3,彼岸领域,"Action, Adventure, Casual",弹炸人2222,2017,"Action, Adventure, Casual",767400,彼岸领域,"Action, Casual, Adventure"
4,Trickjump Games Ltd,"Action, Adventure, Simulation",Battle Royale Trainer,2018,"Action, Adventure, Simulation, FPS, Shooter, T...",772540,Trickjump Games Ltd,"Third-Person Shooter, Sniper, Shooter, Simulat..."
...,...,...,...,...,...,...,...,...
28373,Bidoniera Games,"Action, Adventure, Casual, Indie",Kebab it Up!,2018,"Action, Indie, Casual, Violent, Adventure",745400,Bidoniera Games,"Violent, Casual, Adventure, Action, Indie"
28374,Ghost_RUS Games,"Casual, Indie, Simulation, Strategy",Colony On Mars,2018,"Strategy, Indie, Casual, Simulation",773640,"Nikita ""Ghost_RUS""","Casual, Indie, Simulation, Strategy"
28375,Sacada,"Casual, Indie, Strategy",LOGistICAL: South Africa,2018,"Strategy, Indie, Casual",733530,Sacada,"Casual, Indie, Strategy"
28376,Laush Studio,"Indie, Racing, Simulation",Russian Roads,2018,"Indie, Simulation, Racing",610660,Laush Dmitriy Sergeevich,"Racing, Indie, Simulation"


Ahora que se tiene el nuevo campo se procede a eliminar los originales y a normalizar el nuevo

In [203]:
# Se eliminan las columnas orginales
df_games = df_games.drop(columns='Genres')
df_games = df_games.drop(columns='Tags')
df_games.head(2)

,Publisher,Title,Year,Id,Developer,Genres_Plus
0,Kotoshiro,Lost Summoner Kitty,2018,761140,Kotoshiro,"Simulation, Strategy, Casual, Action, Indie"
1,"Making Fun, Inc.",Ironbound,2018,643980,Secret Level SRL,"2D, Replay Value, RPG, Turn-Based, Free to Pla..."


In [204]:
# Se le cambia el nombre al campo nuevo
df_games = df_games.rename(columns={'Genres_Plus': 'Genres'})
df_games.head(2)

,Publisher,Title,Year,Id,Developer,Genres
0,Kotoshiro,Lost Summoner Kitty,2018,761140,Kotoshiro,"Simulation, Strategy, Casual, Action, Indie"
1,"Making Fun, Inc.",Ironbound,2018,643980,Secret Level SRL,"2D, Replay Value, RPG, Turn-Based, Free to Pla..."


Para una mejor lectura del Data Frame se decide cambiar la posición del campo "Publisher"

In [205]:
columnas = list(df_games.columns)
columnas[0], columnas[3] = columnas[3], columnas[0]
df_games = df_games[columnas]
df_games.head()

,Id,Title,Year,Publisher,Developer,Genres
0,761140,Lost Summoner Kitty,2018,Kotoshiro,Kotoshiro,"Simulation, Strategy, Casual, Action, Indie"
1,643980,Ironbound,2018,"Making Fun, Inc.",Secret Level SRL,"2D, Replay Value, RPG, Turn-Based, Free to Pla..."
2,670290,Real Pool 3D - Poolians,2017,Poolians.com,Poolians.com,"Sports, Simulation, Free to Play, Multiplayer,..."
3,767400,弹炸人2222,2017,彼岸领域,彼岸领域,"Action, Casual, Adventure"
4,772540,Battle Royale Trainer,2018,Trickjump Games Ltd,Trickjump Games Ltd,"Third-Person Shooter, Sniper, Shooter, Simulat..."


### Se guarda el DataFrame en formatos mas ligeros

In [ ]:
df_games.to_csv('user_games.csv', index=False) # Se guarda el DataFrame en formato CSV)

In [ ]:
# Se convierte el archivo CSV a parquet bajo la compresión "gzip"
ruta_csv_items = 'E:\\AAADATOS\\Henry\\AA_Data_Science\\MATERIAL_PI\\PI_ML_OPS_STEAM_DSFT17\\user_games.csv' # Se crea una variable con la ruta del archivo CSV
df_games_temp = pd.read_csv(ruta_csv_items) # Se lee ese CSV en un nuevo DataFrame temporal para segurar como estan los datos
df_games_temp

,User_Id,Items_Count,Steam_Id,Item_Id,Item_Name,Playtime_Forever_Hours
0,76561197970982479,277,76561197970982479,10,Counter-Strike,6
1,76561197970982479,277,76561197970982479,30,Day of Defeat,7
2,76561197970982479,277,76561197970982479,300,Day of Defeat: Source,4733
3,76561197970982479,277,76561197970982479,240,Counter-Strike: Source,1853
4,76561197970982479,277,76561197970982479,3830,Psychonauts,333
...,...,...,...,...,...,...
3285241,76561198329548331,7,76561198329548331,304930,Unturned,677
3285242,76561198329548331,7,76561198329548331,227940,Heroes & Generals,43
3285243,76561198329548331,7,76561198329548331,388490,One Way To Die: Steam Edition,3
3285244,76561198329548331,7,76561198329548331,521570,You Have 10 Seconds 2,4


Se guarada el DataFrame temporal en formato comprimido gzip

In [ ]:
df_games_temp.to_csv('user_games.gzip', compression='gzip', index=False)

In [ ]:
# Se verifica que el archivo gzip comprimido pueda ser leido de manera efectiva y que no hayan problemas en los datos que este arroja
df_games_parquet = pd.read_csv('user_games.gzip', compression='gzip')
df_games_parquet

,User_Id,Items_Count,Steam_Id,Item_Id,Item_Name,Playtime_Forever_Hours
0,76561197970982479,277,76561197970982479,10,Counter-Strike,6
1,76561197970982479,277,76561197970982479,30,Day of Defeat,7
2,76561197970982479,277,76561197970982479,300,Day of Defeat: Source,4733
3,76561197970982479,277,76561197970982479,240,Counter-Strike: Source,1853
4,76561197970982479,277,76561197970982479,3830,Psychonauts,333
...,...,...,...,...,...,...
3285241,76561198329548331,7,76561198329548331,304930,Unturned,677
3285242,76561198329548331,7,76561198329548331,227940,Heroes & Generals,43
3285243,76561198329548331,7,76561198329548331,388490,One Way To Die: Steam Edition,3
3285244,76561198329548331,7,76561198329548331,521570,You Have 10 Seconds 2,4


In [ ]:
df_items_parquet.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3285246 entries, 0 to 3285245
Data columns (total 6 columns):
 #   Column                  Dtype 
---  ------                  ----- 
 0   User_Id                 object
 1   Items_Count             int64 
 2   Steam_Id                int64 
 3   Item_Id                 int64 
 4   Item_Name               object
 5   Playtime_Forever_Hours  int64 
dtypes: int64(4), object(2)
memory usage: 150.4+ MB
